In [1]:
import numpy as np
import grape as utils
from scipy.linalg import expm
import matplotlib.pyplot as plt
import copy
from scipy.optimize import minimize
from scipy.optimize import Bounds

In [4]:
nions=1
max_amp = 2 * np.pi * 25e3  
sx,sy,sz, ssx, ssy, ssz, Had = utils.get_spinops(nions,1/300)
utarg = expm(-1j * (np.pi / 2 ) * sx[0] / 2) 

tau = 40e-6
nsegments = 200
del_t = tau/nsegments 
x0 = np.random.rand(nsegments*2);
# x0 = result.x
# x2 = np.load('x120s_optimized.npy'); 
# x0 = utils.smooth_pulse(x2,60,2)
# x0 = np.load('x30s_optimized.npy'); 
# x0 = np.asarray([-2.94650661e-01,  8.18969715e-01, -8.72864686e-01, -6.51449500e-01,
#         1.00000000e+00,  1.00000000e+00,  1.00000000e+00, -3.05380167e-06,
#        -1.00000000e+00, -1.00000000e+00]) # for no rfi, 5, 12e-6, fun 1.6681861447764845e-07
# x1 = np.repeat(x2,10)
# x0 = utils.smooth_pulse(x1,50,2)

# rfi=[1]+np.linspace(0,1/300,5).tolist()
rfi=[1]
# rfi=[1.01,1,0.99]
print(utils.calc_fidelity6_ct(x0, nsegments, rfi, ssx, ssy, utarg, del_t, nions, max_amp))
bnd = Bounds(lb=-np.ones(nsegments*2),ub=np.ones(nsegments*2))
result = minimize(utils.calc_fidelity6_ct,
         x0=x0,
         jac=utils.calc_grad2,
         args = (nsegments, rfi, ssx, ssy, utarg, del_t, nions, max_amp),
         method='L-BFGS-B ',
         options={'disp':True, 'gtol':1e-16 },
        )
result

4711.4299901977765
Optimization terminated successfully.
         Current function value: -0.000000
         Iterations: 27
         Function evaluations: 95
         Gradient evaluations: 91


  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: -4.440892098500626e-12
        x: [ 1.329e-01 -1.783e-01 ...  3.916e-01 -1.067e-01]
      nit: 27
      jac: [-3.331e-19 -2.220e-19 ...  2.020e-17  2.051e-17]
 hess_inv: [[ 2.409e+00  1.405e+00 ... -4.391e-02 -5.164e-02]
            [ 1.405e+00  2.402e+00 ... -1.723e-02 -2.461e-02]
            ...
            [-4.391e-02 -1.723e-02 ...  1.385e+01  1.297e+01]
            [-5.164e-02 -2.461e-02 ...  1.297e+01  1.409e+01]]
     nfev: 95
     njev: 91

In [5]:
print(utils.calc_fidelity5(result.x , nsegments, np.asarray([[1,1]]) , ssx, ssy, utarg, del_t, nions, max_amp))
print(utils.calc_fidelity5(result.x*1/300 , nsegments, np.asarray([[1,1]]) , ssx, ssy, np.eye(2), del_t, nions, max_amp))


0.0
0.0344269677088338


In [14]:
nions=1
max_amp = 2 * np.pi * 25e3  
sx,sy,sz, ssx, ssy, ssz, Had = utils.get_spinops(nions,1/300)
utarg = expm(-1j * (np.pi / 2 ) * sx[0] / 2)
rffull = np.linspace(0,1/300,50); rffull[0]=1

nsegments = 120
del_t = 0.25e-6

numrfi_add = 2
for j in range(1,10):
        if j==1:
                # x0 = np.load('x120s_optimized.npy') 
                x0 = result.x 
        else:
                x0 = result.x
                
        rfi=rffull[0:numrfi_add*j+1].tolist()
        print(f"running for {j}")
        print(rfi)
        result = minimize(utils.calc_fidelity6_ct,
         x0=x0,
         jac=utils.calc_grad2,
         args = (nsegments, rfi, ssx, ssy, utarg, del_t, nions, max_amp),
         method='BFGS',
         options={'disp':True, 'gtol':1e-9},
        )
        result

running for 1
[1.0, 6.802721088435374e-05, 0.00013605442176870748]
         Current function value: 0.000024
         Iterations: 0
         Function evaluations: 46
         Gradient evaluations: 34


/Users/katiyar/opt/anaconda3/envs/oneQgate/lib/python3.12/site-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


In [35]:
# one point optimization
print(utils.calc_fidelity6_ct(result.x, nsegments, [1], ssx, ssy, utarg, del_t, nions, max_amp))
print(utils.calc_fidelity6_ct(result.x, nsegments, [1/300], ssx, ssy, utarg, del_t, nions, max_amp))


2.220446049250313e-12
0.002052226979465388


In [51]:
# two point optimization
print(utils.calc_fidelity6_ct(result.x, nsegments, [1], ssx, ssy, utarg, del_t, nions, max_amp))
print(utils.calc_fidelity6_ct(result.x, nsegments, [1/300], ssx, ssy, utarg, del_t, nions, max_amp))

2.1122336556955723
0.7895345174480095


In [16]:
xx = np.load('x120s_optimized.npy')
print(utils.calc_fidelity6_ct(xx, nsegments, [1/300], ssx, ssy, utarg, del_t, nions, max_amp))


0.13166070319314827
